In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
!pip install unsloth vllm bitsandbytes peft accelerate trl

In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

# 1. Load Model (4-bit saves VRAM)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-1.5B-Instruct",
    load_in_4bit = True,
)

# 2. Add LoRA Adapters
model = FastLanguageModel.get_peft_model(model, r = 16, target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"])

# 3. Load Dataset
dataset = load_dataset("json", data_files="dataset.jsonl", split="train")
# 4. Define the Alpaca prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN or the model won't stop talking

# 5. Define the formatting function
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# 6. Map the function to your dataset
dataset = dataset.map(formatting_prompts_func, batched = True)
# 7. Run Trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", # This tells the trainer to look at our new "glued" text
    max_seq_length = 2048,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # 60 steps is perfect for a quick test
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# Start the training!
trainer_stats = trainer.train()

The final exam. Testing the model

In [ ]:
# 1. Enable 2x faster inference
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)

# 2. Pick a sentence the model has NEVER seen before
test_sentence = "I see a bright orange motorized ball valve on line 702."

# 3. Use the EXACT same template from training
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Extract equipment asset details from the sentence into a structured JSON object.", # Instruction
        test_sentence, # Input
        "", # Leave Response blank for the model to fill
    )
], return_tensors = "pt").to("cuda")

# 4. Generate the answer
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)

# 5. Decode and show the result
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


{"tag": "VLV", "type": "ball", "op": "motorized", "color": "orange", "line": 702}<|im_end|>


In [ ]:
# Save the model in the GGUF format for Ollama
model.save_pretrained_gguf("my_asset_model", tokenizer, quantization_method = "q4_k_m")